哈希表可以理解为一个加强版的数组。

数组可以通过索引在 O(1) 的时间复杂度内查找到对应元素，索引是一个非负整数。

哈希表是类似的，可以通过 key 在 O(1) 的时间复杂度内查找到这个 key 对应的 value。key 的类型可以是数字、字符串等多种类型。

怎么做的？特别简单，哈希表的底层实现就是一个数组（我们不妨称之为 table）。它先把这个 key 通过一个哈希函数（我们不妨称之为 hash）转化成数组里面的索引，然后增删查改操作和数组基本相同：

In [ ]:
# 哈希表伪码逻辑
class MyHashMap:

    def __init__(self):
        self.table = [None] * 1000

    # 增/改，复杂度 O(1)
    def put(self, key, value):
        index = self.hash(key)
        self.table[index] = value

    # 查，复杂度 O(1)
    def get(self, key):
        index = self.hash(key)
        return self.table[index]

    # 删，复杂度 O(1)
    def remove(self, key):
        index = self.hash(key)
        self.table[index] = None

    # 哈希函数，把 key 转化成 table 中的合法索引
    # 时间复杂度必须是 O(1)，才能保证上述方法的复杂度都是 O(1)
    def hash(self, key):
        # ...
        pass

## 扩容和负载因子

拉链法和线性探查法虽然能解决哈希冲突的问题，但是它们会导致性能下降。

比如拉链法，你算出来 index = hash(key) 这个索引了，结果过去查出来的是个链表，你还得遍历一下这个链表，才能在里面找到你要的 value。这个过程的时间复杂度是 O(K)，K 是这个链表的长度。

线性探查法也是类似的，你算出来 index = hash(key) 这个索引了，你去这个索引位置查看，发现存储的不是要找的 key，但由于线性探查法解决哈希冲突的方式，你并不能确定这个 key 真的不存在，你必须顺着这个索引往后找，直到找到一个空的位置或者找到这个 key 为止，这个过程的时间复杂度也是O(K)，K 为连续探查的次数。

所以说，如果频繁出现哈希冲突，那么 K 的值就会增大，这个哈希表的性能就会显著下降。这是我们需要避免的。

那么为什么会频繁出现哈希冲突呢？两个原因呗：

1、哈希函数设计的不好，导致 key 的哈希值分布不均匀，很多 key 映射到了同一个索引上。

2、哈希表里面已经装了太多的 key-value 对了，这种情况下即使哈希函数再完美，也没办法避免哈希冲突。

对于第一个问题没什么好说的，开发编程语言标准库的大佬们已经帮你设计好了哈希函数，你只要调用就行了。

对于第二个问题是我们可以控制的，即避免哈希表装太满，这就引出了「负载因子」的概念。

负载因子是一个哈希表装满的程度的度量。一般来说，负载因子越大，说明哈希表里面存储的 key-value 对越多，哈希冲突的概率就越大，哈希表的操作性能就越差。

负载因子的计算公式也很简单，就是 size / table.length。其中 size 是哈希表里面的 key-value 对的数量，table.length 是哈希表底层数组的容量。

你不难发现，用拉链法实现的哈希表，负载因子可以无限大，因为链表可以无限延伸；用线性探查法实现的哈希表，负载因子不会超过 1。

像 Java 的 HashMap，允许我们创建哈希表时自定义负载因子，不设置的话默认是 0.75，这个值是经验值，一般保持默认就行了。

当哈希表内元素达到负载因子时，哈希表会扩容。和之前讲解 动态数组的实现 是类似的，就是把哈希表底层 table 数组的容量扩大，把数据搬移到新的大数组中。size 不变，table.length 增加，负载因子就减小了。


## 为什么不能依赖哈希表的遍历顺序

哈希表的遍历本质上就是遍历那个底层 table 数组

首先，由于 hash 函数要把你的 key 进行映射，所以 key 在底层 table 数组中的分布是随机的，不像数组/链表结构那样有个明确的元素顺序。

其次，刚才讲了哈希表达到负载因子时会怎样？会扩容对吧，也就是 table.length 会变化，且会搬移元素。

那么这个搬移数据的过程，是不是要用 hash 函数重新计算 key 的哈希值，然后放到新的 table 数组中？

而这个 hash 函数，它计算出的值依赖 table.length。也就是说，哈希表自动扩容后，同一个 key 的哈希值可能变化，即这个 key-value 对儿存储在 table 的索引也变了，所以遍历结果的顺序就和之前不一样了。

你观察到的现象就是，这次遍历的第一个键是 key1，但是增删几个元素再遍历，可能发现 key1 跑到最后去了。

## 拉链法实现哈希表

拉链法相当于是哈希表的底层数组并不直接存储 value 类型，而是存储一个链表，当有多个不同的 key 映射到了同一个索引上，这些 key -> value 对儿就存储在这个链表中，这样就能解决哈希冲突的问题。

In [ ]:
class ListNode:
    def __init__(self, key=None, value=None):
        self.key = key
        self.value = value
        self.next = None


class MyHashMapLink:

    def __init__(self):
        self.size = 1000
        self.table = [None] * self.size

    def put(self, key, value):
        index = self.hash(key)
        if self.table[index] is None:
            self.table[index] = ListNode(key, value)
        else:
            current = self.table[index]
            while True:
                if current.key == key:
                    current.value = value
                    return
                if current.next is None:
                    break
                current = current.next
            current.next = ListNode(key, value)

    def get(self, key):
        index = self.hash(key)
        current = self.table[index]
        while current:
            if current.key == key:
                return current.value
            current = current.next
        return -1

    def remove(self, key):
        index = self.hash(key)
        current = self.table[index]
        if not current:
            return
        if current.key == key:
            self.table[index] = current.next
        else:
            prev = current
            current = current.next
            while current:
                if current.key == key:
                    prev.next = current.next
                    return
                prev = current
                current = current.next

    def hash(self, key):
        return key % self.size

## 线性探查法（开放寻址法）实现哈希表

线性探查法的思路是，一个 key 发现算出来的 index 值已经被别的 key 占了，那么它就去 index + 1 的位置看看，如果还是被占了，就继续往后找，直到找到一个空的位置为止

In [ ]:
class MyHashMapLinearProbing:

    def __init__(self):
        self.size = 1000
        self.table = [None] * self.size

    def put(self, key, value):
        index = self.hash(key)
        # 向后查
        while self.table[index] is not None and self.table[index][0] != key:
            index = (index + 1) % self.size
        self.table[index] = (key, value)

    def get(self, key):
        index = self.hash(key)
        while self.table[index] is not None:
            if self.table[index][0] == key:
                return self.table[index][1]
            index = (index + 1) % self.size
        return -1

    def remove(self, key):
        index = self.hash(key)
        while self.table[index] is not None:
            if self.table[index][0] == key:
                self.table[index] = None
                return
            index = (index + 1) % self.size

    def hash(self, key):
        return key % self.size

## 哈希集合

哈希表的键，其实就是哈希集合。

In [ ]:
class MyHashSet:

    def __init__(self):
        self.size = 1000
        self.table = [None] * self.size

    def add(self, key):
        index = self.hash(key)
        if self.table[index] is None:
            self.table[index] = [key]
        else:
            if key not in self.table[index]:
                self.table[index].append(key)

    def remove(self, key):
        index = self.hash(key)
        if self.table[index] is not None:
            while key in self.table[index]:
                self.table[index].remove(key)

    def contains(self, key):
        index = self.hash(key)
        if self.table[index] is not None:
            return key in self.table[index]
        return False

    def hash(self, key):
        return key % self.size

## 哈希链表

前面说过，哈希表中的key是无序的，但是有的时候我们又需要获得有序的key，我们可以使用`双链表`对哈希表进行加强，让哈希表的键保持插入顺序

In [ ]:
class ListNode:
    def __init__(self, key=None, value=None):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LinkedHashMap:
    def __init__(self):
        self.map = {}
        self.head = ListNode()
        self.tail = ListNode()
        self.head.next = self.tail
        self.tail.prev = self.head

    def _add_node(self, node):
        node.prev = self.tail.prev
        node.next = self.tail
        self.tail.prev.next = node
        self.tail.prev = node

    def _remove_node(self, node):
        prev = node.prev
        next = node.next
        prev.next = next
        next.prev = prev

    def put(self, key, value):
        if key in self.map:
            self._remove_node(self.map[key])
        new_node = ListNode(key, value)
        self._add_node(new_node)
        self.map[key] = new_node

    def get(self, key):
        if key in self.map:
            node = self.map[key]
            return node.value
        return -1

    def remove(self, key):
        if key in self.map:
            self._remove_node(self.map[key])
            del self.map[key]

    def __iter__(self):
        current = self.head.next
        while current != self.tail:
            yield current.key, current.value
            current = current.next